## Import libraries

In [ ]:
import numpy as np
from seismic_zfp.read import SgzReader
import seismic_zfp
import matplotlib
import matplotlib.pyplot as plt

### Seismic

In [ ]:
seismic_filename = '/Users/ruslan/Downloads/Ichthys 3D seismic for fault competition.sgz'
# filename for z-optimized volume
seismic_z_filename = '/Users/ruslan/Downloads/Ichthys 3D seismic for fault competition_z.sgz'
file = SgzReader(seismic_filename)
file_z = SgzReader(seismic_z_filename)

### Overlay data (fault cube)

In [ ]:
flt_filename = '/Users/ruslan/Downloads/faults_geoplat.sgz'
flt_z_filename = '/Users/ruslan/Downloads/faults_geoplat_z.sgz'
flt_file = SgzReader(flt_filename)
flt_file_z = SgzReader(flt_z_filename)

## Plot vertical slice with Matplotlib

In [ ]:
# define the range for visualization
t = seismic_zfp.tools.cube(seismic_filename)
vm = np.percentile(t, 95)
print(vm)
del t

In [ ]:
# Plot the central inline in the usual orientation by transposing the data
plt.imshow(file.read_inline(file.n_ilines//2).T, cmap='gray', aspect='auto', vmin=-vm, vmax=vm)

plt.xlabel('XLINE_indx')
plt.ylabel('TIME_indx')
plt.title('Ichthys 3D -INL '+ str(file.n_ilines//2))
plt.show()

## Plot time slice with Matplotlib

In [ ]:
plt.imshow(file.read_zslice(file.n_samples//2).T, vmin=-vm, vmax=vm, cmap='gray', origin='lower', aspect='auto')

plt.xlabel('XLINE_indx')
plt.ylabel('INLINE_indx')
plt.title('Time Slice '+ str(file.n_samples//2))
plt.show()

###  Plotting with HOLOVIZ

In [ ]:
import hvplot.xarray
import panel as pn
import xarray as xr
from holoviews import opts
import holoviews as hv
hv.extension('bokeh')

opts.defaults(
    opts.Image(
        width=900, height=700), tools=["layers_control"])

### Interactive functions

In [ ]:
def get_flt_mask(attr, eps):
    flt_mask = np.zeros([attr.shape[0], attr.shape[1], 4])
    flt_mask[:, :, 0][attr > eps] = 1
    flt_mask[:, :, 3][attr > eps] = 1
    return hv.RGB(flt_mask)

def plot_inl(inl, cmap, vmin, vmax, eps): 
    seis = file.read_inline(inl).T
    attr = flt_file.read_inline(inl).T
    seis_plot = hv.Image(seis).opts(opts.Image(clim=(-vmin, vmax), cmap=cmap, xlabel='Xline_idx', ylabel='Time_idx'))
    rgba = get_flt_mask(attr, eps)
    return seis_plot*rgba
def plot_xln(xln, cmap, vmin, vmax, eps): 
    seis = file.read_crossline(xln).T
    attr = flt_file.read_crossline(xln).T
    seis_plot = hv.Image(seis).opts(opts.Image(clim=(-vmin, vmax), cmap=cmap, xlabel='Iline_idx', ylabel='Time_idx'))
    rgba = get_flt_mask(attr, eps)
    return seis_plot*rgba
   
def plot_top(t, cmap, vmin, vmax, eps):
    seis = file_z.read_zslice(t).T
    attr = flt_file_z.read_zslice(t).T
    seis_plot = hv.Image(seis).opts(opts.Image(clim=(-vmin, vmax), cmap=cmap, xlabel='Xline_idx', ylabel='Iline_idx'))
    rgba = get_flt_mask(attr, eps)
    return seis_plot*rgba

iline_old = 0
xline_old = 0
t_old = 0
    
def plot_slice_seis(inline_indx, xline_indx, time_indx, cmap, eps):
    global iline_old, xline_old, t_old
    if np.abs(iline_old - inline_indx) > 0:
        iline_old = inline_indx
        return plot_inl(inline_indx, cmap, vm, vm, eps)
    elif np.abs(xline_old - xline_indx) > 0:
        xline_old = xline_indx
        return plot_xln(xline_indx, cmap, vm, vm, eps)
    elif np.abs(t_old - time_indx) > 0:
        t_old = time_indx
        return plot_top(time_indx, cmap, vm, vm, eps)

### Layout

In [ ]:
pallet_select = pn.widgets.Select(name='Pallet', options=['gray', 'seismic', 'bwr'])  
float_slider = pn.widgets.FloatSlider(name='Probability Threshold', start=0, end=1.0, step=0.1, value=0.5)
inline_slider = pn.widgets.IntSlider(name='Inline_indx', start=0, end=file.n_ilines-1)
xline_slider = pn.widgets.IntSlider(name='Xline_indx', start=0, end=file.n_xlines-1)
tline_slider = pn.widgets.IntSlider(name='Time_indx', start=0, end=file.n_samples-1)
col = pn.Column(pn.Row (pallet_select,float_slider), pn.Row (inline_slider, xline_slider, tline_slider))
layout_seis = pn.interact(plot_slice_seis, inline_indx = inline_slider, xline_indx=xline_slider, time_indx = tline_slider, cmap=pallet_select, eps=float_slider )

In [ ]:
pn.Column(col,  layout_seis[1])